# Site Data Analysis and Metadata Generation

This notebook analyzes all site data files and generates comprehensive metadata including:
- Date ranges and year coverage
- Missing values analysis
- Data quality checks
- Statistical summaries

In [5]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [6]:
def analyze_csv_file(file_path, file_name):
    """
    Analyze a single CSV file and extract metadata.
    
    Parameters:
    -----------
    file_path : str
        Full path to the CSV file
    file_name : str
        Name of the CSV file
    
    Returns:
    --------
    dict : Dictionary containing file metadata
    """
    print(f"\n  Analyzing: {file_name}")
    print(f"  {'-'*76}")
    
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Basic file information
        metadata = {
            'file_name': file_name,
            'file_path': file_path,
            'total_rows': len(df),
            'total_columns': len(df.columns),
            'columns': list(df.columns),
            'dtypes': df.dtypes.to_dict(),
            'file_size_mb': os.path.getsize(file_path) / (1024 * 1024)
        }
        
        # Date and time analysis
        if 'day' in df.columns:
            df['day'] = pd.to_datetime(df['day'])
            metadata['date_range'] = {
                'start_date': df['day'].min().strftime('%Y-%m-%d'),
                'end_date': df['day'].max().strftime('%Y-%m-%d'),
                'start_datetime': df['day'].min(),
                'end_datetime': df['day'].max(),
                'total_days': (df['day'].max() - df['day'].min()).days + 1,
                'years': sorted(df['day'].dt.year.unique().tolist()),
                'year_range': f"{df['day'].min().year} - {df['day'].max().year}",
                'unique_dates': df['day'].nunique()
            }
        
        # Missing values analysis
        missing_values = df.isnull().sum()
        missing_percent = (missing_values / len(df)) * 100
        
        metadata['missing_values'] = {
            'total_missing': missing_values.sum(),
            'columns_with_missing': missing_values[missing_values > 0].to_dict(),
            'missing_percentage': missing_percent[missing_percent > 0].to_dict(),
            'complete_rows': len(df) - df.isnull().any(axis=1).sum(),
            'rows_with_missing': df.isnull().any(axis=1).sum()
        }
        
        # Statistical summary for numeric columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if numeric_cols:
            metadata['statistics'] = df[numeric_cols].describe().to_dict()
            metadata['numeric_columns'] = numeric_cols
        
        # Data quality checks
        metadata['quality_checks'] = {
            'duplicate_rows': df.duplicated().sum(),
            'negative_values': {},
            'zero_values': {},
            'outliers_detected': {}
        }
        
        # Check for negative values in numeric columns
        for col in numeric_cols:
            negative_count = (df[col] < 0).sum()
            zero_count = (df[col] == 0).sum()
            if negative_count > 0:
                metadata['quality_checks']['negative_values'][col] = negative_count
            if zero_count > 0:
                metadata['quality_checks']['zero_values'][col] = zero_count
            
            # Simple outlier detection (values beyond 3 standard deviations)
            if df[col].std() > 0:
                mean_val = df[col].mean()
                std_val = df[col].std()
                outliers = ((df[col] < mean_val - 3*std_val) | (df[col] > mean_val + 3*std_val)).sum()
                if outliers > 0:
                    metadata['quality_checks']['outliers_detected'][col] = outliers
        
        # Time-related analysis
        if all(col in df.columns for col in ['hour', 'minute']):
            metadata['time_analysis'] = {
                'hour_range': f"{df['hour'].min()} - {df['hour'].max()}",
                'minute_range': f"{df['minute'].min()} - {df['minute'].max()}",
                'unique_hours': sorted(df['hour'].unique().tolist()),
                'unique_minutes': sorted(df['minute'].unique().tolist()),
                'expected_records_per_day': df.groupby('day').size().describe().to_dict() if 'day' in df.columns else None
            }
        
        # Print summary
        print(f"    ✓ Rows: {metadata['total_rows']:,}")
        print(f"    ✓ Columns: {metadata['total_columns']}")
        if 'date_range' in metadata:
            print(f"    ✓ Date Range: {metadata['date_range']['start_date']} to {metadata['date_range']['end_date']}")
            print(f"    ✓ Years: {metadata['date_range']['year_range']}")
        print(f"    ✓ Missing Values: {metadata['missing_values']['total_missing']:,} ({metadata['missing_values']['total_missing']/len(df)*100:.2f}%)")
        print(f"    ✓ Duplicate Rows: {metadata['quality_checks']['duplicate_rows']:,}")
        
        return metadata
        
    except Exception as e:
        print(f"    ✗ ERROR analyzing {file_name}: {str(e)}")
        return {
            'file_name': file_name,
            'error': str(e)
        }

In [7]:
def aggregate_site_summary(files_metadata):
    """
    Aggregate metadata across all files in a site.
    
    Parameters:
    -----------
    files_metadata : dict
        Dictionary of file metadata
    
    Returns:
    --------
    dict : Aggregated summary statistics
    """
    summary = {
        'total_files': len(files_metadata),
        'total_rows': 0,
        'total_missing_values': 0,
        'all_years': set(),
        'date_range_all': {'start': None, 'end': None},
        'all_columns': set()
    }
    
    for file_name, file_meta in files_metadata.items():
        if 'error' in file_meta:
            continue
            
        summary['total_rows'] += file_meta.get('total_rows', 0)
        summary['total_missing_values'] += file_meta.get('missing_values', {}).get('total_missing', 0)
        
        if 'date_range' in file_meta:
            years = file_meta['date_range'].get('years', [])
            summary['all_years'].update(years)
            
            start = file_meta['date_range'].get('start_datetime')
            end = file_meta['date_range'].get('end_datetime')
            
            if start and (summary['date_range_all']['start'] is None or start < summary['date_range_all']['start']):
                summary['date_range_all']['start'] = start
            if end and (summary['date_range_all']['end'] is None or end > summary['date_range_all']['end']):
                summary['date_range_all']['end'] = end
        
        if 'columns' in file_meta:
            summary['all_columns'].update(file_meta['columns'])
    
    summary['all_years'] = sorted(list(summary['all_years']))
    summary['year_range'] = f"{min(summary['all_years'])} - {max(summary['all_years'])}" if summary['all_years'] else "N/A"
    
    if summary['date_range_all']['start']:
        summary['date_range_all']['start_str'] = summary['date_range_all']['start'].strftime('%Y-%m-%d')
        summary['date_range_all']['end_str'] = summary['date_range_all']['end'].strftime('%Y-%m-%d')
        summary['date_range_all']['total_days'] = (summary['date_range_all']['end'] - summary['date_range_all']['start']).days + 1
    
    summary['all_columns'] = sorted(list(summary['all_columns']))
    
    return summary

In [2]:
def analyze_site_data(site_path):
    """
    Analyze all CSV files in a site directory and generate comprehensive metadata.
    
    Parameters:
    -----------
    site_path : str
        Path to the site directory (e.g., 'Site Data/Site 01')
    
    Returns:
    --------
    dict : Dictionary containing metadata for all files in the site
    """
    site_name = os.path.basename(site_path)
    site_metadata = {
        'site_name': site_name,
        'files': {}
    }
    
    # Get all CSV files in the site directory
    csv_files = sorted([f for f in os.listdir(site_path) if f.endswith('.csv')])
    
    print(f"\n{'='*80}")
    print(f"ANALYZING: {site_name}")
    print(f"{'='*80}")
    
    for csv_file in csv_files:
        file_path = os.path.join(site_path, csv_file)
        file_metadata = analyze_csv_file(file_path, csv_file)
        site_metadata['files'][csv_file] = file_metadata
    
    # Aggregate site-level statistics
    site_metadata['summary'] = aggregate_site_summary(site_metadata['files'])
    
    return site_metadata

In [3]:
def print_site_summary(site_metadata):
    """
    Print a formatted summary of site metadata.
    
    Parameters:
    -----------
    site_metadata : dict
        Site metadata dictionary
    """
    summary = site_metadata['summary']
    
    print(f"\n{'='*80}")
    print(f"SITE SUMMARY: {site_metadata['site_name']}")
    print(f"{'='*80}")
    print(f"Total Files: {summary['total_files']}")
    print(f"Total Rows: {summary['total_rows']:,}")
    print(f"Total Missing Values: {summary['total_missing_values']:,}")
    
    if summary['date_range_all']['start']:
        print(f"\nDate Range (All Files):")
        print(f"  Start: {summary['date_range_all']['start_str']}")
        print(f"  End:   {summary['date_range_all']['end_str']}")
        print(f"  Total Days: {summary['date_range_all']['total_days']:,}")
    
    if summary['all_years']:
        print(f"\nYear Coverage: {summary['year_range']}")
        print(f"  Years: {', '.join(map(str, summary['all_years']))}")
    
    print(f"\nAll Columns Found: {', '.join(summary['all_columns'])}")
    print(f"{'='*80}\n")

## Analyze All Sites

In [8]:
# Set the base path to Site Data folder
base_path = "/Users/IRFAN/Desktop/Irradiance-forecasting/Site Data"

# Get all site directories
site_dirs = sorted([d for d in os.listdir(base_path) 
                   if os.path.isdir(os.path.join(base_path, d)) and d.startswith('Site')])

print(f"Found {len(site_dirs)} site(s) to analyze:")
for site_dir in site_dirs:
    print(f"  - {site_dir}")

# Store all site metadata
all_sites_metadata = {}

Found 3 site(s) to analyze:
  - Site 01
  - Site 02
  - Site 03


In [9]:
# Analyze each site
for site_dir in site_dirs:
    site_path = os.path.join(base_path, site_dir)
    site_metadata = analyze_site_data(site_path)
    all_sites_metadata[site_dir] = site_metadata
    print_site_summary(site_metadata)


ANALYZING: Site 01

  Analyzing: Q01.csv
  ----------------------------------------------------------------------------
    ✓ Rows: 122,724
    ✓ Columns: 6
    ✓ Date Range: 2024-04-05 to 2024-06-30
    ✓ Years: 2024 - 2024
    ✓ Missing Values: 60 (0.05%)
    ✓ Duplicate Rows: 0

  Analyzing: Q02.csv
  ----------------------------------------------------------------------------
    ✓ Rows: 131,528
    ✓ Columns: 6
    ✓ Date Range: 2024-07-01 to 2024-09-30
    ✓ Years: 2024 - 2024
    ✓ Missing Values: 14 (0.01%)
    ✓ Duplicate Rows: 4

  Analyzing: Q03.csv
  ----------------------------------------------------------------------------
    ✓ Rows: 131,852
    ✓ Columns: 6
    ✓ Date Range: 2024-10-01 to 2024-12-31
    ✓ Years: 2024 - 2024
    ✓ Missing Values: 52 (0.04%)
    ✓ Duplicate Rows: 0

  Analyzing: Q04.csv
  ----------------------------------------------------------------------------
    ✓ Rows: 170,689
    ✓ Columns: 6
    ✓ Date Range: 2025-01-01 to 2025-04-30
    ✓ Year

## Detailed Metadata Access

You can access detailed metadata for any site or file using the `all_sites_metadata` dictionary.

In [10]:
# Example: View detailed metadata for Site 01, Q01.csv
if 'Site 01' in all_sites_metadata and 'Q01.csv' in all_sites_metadata['Site 01']['files']:
    import json
    q01_meta = all_sites_metadata['Site 01']['files']['Q01.csv']
    print("Detailed Metadata for Site 01 - Q01.csv:")
    print(json.dumps(q01_meta, indent=2, default=str))

Detailed Metadata for Site 01 - Q01.csv:
{
  "file_name": "Q01.csv",
  "file_path": "/Users/IRFAN/Desktop/Irradiance-forecasting/Site Data/Site 01/Q01.csv",
  "total_rows": 122724,
  "total_columns": 6,
  "columns": [
    "item_id",
    "day",
    "hour",
    "minute",
    "w_ghr",
    "w_gtr"
  ],
  "dtypes": {
    "item_id": "object",
    "day": "object",
    "hour": "int64",
    "minute": "int64",
    "w_ghr": "float64",
    "w_gtr": "float64"
  },
  "file_size_mb": 3.5459346771240234,
  "date_range": {
    "start_date": "2024-04-05",
    "end_date": "2024-06-30",
    "start_datetime": "2024-04-05 00:00:00",
    "end_datetime": "2024-06-30 00:00:00",
    "total_days": 87,
    "years": [
      2024
    ],
    "year_range": "2024 - 2024",
    "unique_dates": 87
  },
  "missing_values": {
    "total_missing": "60",
    "columns_with_missing": {
      "w_ghr": 30,
      "w_gtr": 30
    },
    "missing_percentage": {
      "w_ghr": 0.024445096313679478,
      "w_gtr": 0.02444509631367947

## Visualize Data Overview

In [11]:
# Create a summary DataFrame for all sites
summary_data = []

for site_name, site_meta in all_sites_metadata.items():
    summary = site_meta['summary']
    summary_data.append({
        'Site': site_name,
        'Total Files': summary['total_files'],
        'Total Rows': summary['total_rows'],
        'Total Missing': summary['total_missing_values'],
        'Year Range': summary.get('year_range', 'N/A'),
        'Date Range': f"{summary['date_range_all'].get('start_str', 'N/A')} to {summary['date_range_all'].get('end_str', 'N/A')}"
    })

summary_df = pd.DataFrame(summary_data)
print("\nSummary Across All Sites:")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)


Summary Across All Sites:
   Site  Total Files  Total Rows  Total Missing  Year Range               Date Range
Site 01            4      556793            150 2024 - 2025 2024-04-05 to 2025-04-30
Site 02            4      567784          22950 2024 - 2025 2024-04-01 to 2025-04-30
Site 03            4      558937              0 2024 - 2025 2024-04-01 to 2025-05-13


In [12]:
# Create detailed file-level summary
file_summary_data = []

for site_name, site_meta in all_sites_metadata.items():
    for file_name, file_meta in site_meta['files'].items():
        if 'error' not in file_meta:
            file_summary_data.append({
                'Site': site_name,
                'File': file_name,
                'Rows': file_meta.get('total_rows', 0),
                'Missing Values': file_meta.get('missing_values', {}).get('total_missing', 0),
                'Date Range': file_meta.get('date_range', {}).get('year_range', 'N/A'),
                'Years': ', '.join(map(str, file_meta.get('date_range', {}).get('years', []))),
                'Duplicate Rows': file_meta.get('quality_checks', {}).get('duplicate_rows', 0)
            })

file_summary_df = pd.DataFrame(file_summary_data)
print("\nDetailed File-Level Summary:")
print("="*100)
print(file_summary_df.to_string(index=False))
print("="*100)


Detailed File-Level Summary:
   Site    File   Rows  Missing Values  Date Range Years  Duplicate Rows
Site 01 Q01.csv 122724              60 2024 - 2024  2024               0
Site 01 Q02.csv 131528              14 2024 - 2024  2024               4
Site 01 Q03.csv 131852              52 2024 - 2024  2024               0
Site 01 Q04.csv 170689              24 2025 - 2025  2025               0
Site 02 Q01.csv 130992              10 2024 - 2024  2024               0
Site 02 Q02.csv 132305           21771 2024 - 2024  2024               0
Site 02 Q03.csv 132111            1105 2024 - 2024  2024               0
Site 02 Q04.csv 172376              64 2025 - 2025  2025               0
Site 03 Q01.csv 112801               0 2024 - 2024  2024               0
Site 03 Q02.csv 130543               0 2024 - 2024  2024               0
Site 03 Q03.csv 129796               0 2024 - 2024  2024               0
Site 03 Q04.csv 185797               0 2025 - 2025  2025               0


## Missing Values Analysis

In [13]:
# Detailed missing values analysis
print("\nMissing Values Analysis by Site and File:")
print("="*100)

for site_name, site_meta in all_sites_metadata.items():
    print(f"\n{site_name}:")
    print("-" * 100)
    for file_name, file_meta in site_meta['files'].items():
        if 'error' not in file_meta:
            missing_info = file_meta.get('missing_values', {})
            if missing_info.get('total_missing', 0) > 0:
                print(f"  {file_name}:")
                print(f"    Total Missing: {missing_info['total_missing']:,}")
                if missing_info.get('columns_with_missing'):
                    for col, count in missing_info['columns_with_missing'].items():
                        pct = missing_info['missing_percentage'].get(col, 0)
                        print(f"      - {col}: {count:,} ({pct:.2f}%)")
            else:
                print(f"  {file_name}: No missing values ✓")


Missing Values Analysis by Site and File:

Site 01:
----------------------------------------------------------------------------------------------------
  Q01.csv:
    Total Missing: 60
      - w_ghr: 30 (0.02%)
      - w_gtr: 30 (0.02%)
  Q02.csv:
    Total Missing: 14
      - w_ghr: 7 (0.01%)
      - w_gtr: 7 (0.01%)
  Q03.csv:
    Total Missing: 52
      - w_ghr: 26 (0.02%)
      - w_gtr: 26 (0.02%)
  Q04.csv:
    Total Missing: 24
      - w_ghr: 12 (0.01%)
      - w_gtr: 12 (0.01%)

Site 02:
----------------------------------------------------------------------------------------------------
  Q01.csv:
    Total Missing: 10
      - w_ghr: 5 (0.00%)
      - w_gtr: 5 (0.00%)
  Q02.csv:
    Total Missing: 21,771
      - w_ghr: 21,760 (16.45%)
      - w_gtr: 11 (0.01%)
  Q03.csv:
    Total Missing: 1,105
      - w_ghr: 1,065 (0.81%)
      - w_gtr: 40 (0.03%)
  Q04.csv:
    Total Missing: 64
      - w_ghr: 32 (0.02%)
      - w_gtr: 32 (0.02%)

Site 03:
----------------------------------

## Load and Preview Sample Data

In [14]:
# Load and display sample data from each site
for site_name, site_meta in all_sites_metadata.items():
    print(f"\n{'='*80}")
    print(f"Sample Data from {site_name}")
    print(f"{'='*80}")
    
    # Get the first file from each site
    first_file = sorted(site_meta['files'].keys())[0] if site_meta['files'] else None
    if first_file and 'error' not in site_meta['files'][first_file]:
        file_path = site_meta['files'][first_file]['file_path']
        df_sample = pd.read_csv(file_path, nrows=10)
        print(f"\nFirst 10 rows from {first_file}:")
        print(df_sample.to_string(index=False))
        print(f"\nData types:")
        print(df_sample.dtypes)
        print(f"\nShape: {df_sample.shape[0]} rows × {df_sample.shape[1]} columns")


Sample Data from Site 01

First 10 rows from Q01.csv:
item_id        day  hour  minute  w_ghr  w_gtr
    WMS 2024-04-05    15       0  813.0  873.0
    WMS 2024-04-05    15       1  795.0  857.0
    WMS 2024-04-05    15       2  799.0  860.0
    WMS 2024-04-05    15       3  808.0  869.0
    WMS 2024-04-05    15       4  814.0  873.0
    WMS 2024-04-05    15       5  800.0  862.0
    WMS 2024-04-05    15       6  794.0  860.0
    WMS 2024-04-05    15       7  793.0  856.0
    WMS 2024-04-05    15       8  778.0  843.0
    WMS 2024-04-05    15       9  773.0  842.0

Data types:
item_id     object
day         object
hour         int64
minute       int64
w_ghr      float64
w_gtr      float64
dtype: object

Shape: 10 rows × 6 columns

Sample Data from Site 02

First 10 rows from Q01.csv:
item_id        day  hour  minute  w_ghr  w_gtr
    WMS 2024-04-01     0       0   -3.0   -4.0
    WMS 2024-04-01     0       1   -3.0   -4.0
    WMS 2024-04-01     0       2   -3.0   -4.0
    WMS 2024-04-

## Time Interval Analysis

In [15]:
# Analyze time interval in the data
print("="*80)
print("TIME INTERVAL ANALYSIS")
print("="*80)

# Check a sample file to determine time interval
sample_file = "/Users/IRFAN/Desktop/Irradiance-forecasting/Site Data/Site 01/Q01.csv"
df_sample = pd.read_csv(sample_file, nrows=200)

# Create datetime column
df_sample['day'] = pd.to_datetime(df_sample['day'])
df_sample['datetime'] = pd.to_datetime(
    df_sample['day'].astype(str) + ' ' + 
    df_sample['hour'].astype(str) + ':' + 
    df_sample['minute'].astype(str).str.zfill(2)
)

# Calculate time differences
df_sample['time_diff'] = df_sample['datetime'].diff()

# Get the most common time difference
most_common_diff = df_sample['time_diff'].mode()[0] if len(df_sample['time_diff'].mode()) > 0 else None

print(f"\nSample Data Structure:")
print(f"  Columns: {', '.join(df_sample.columns)}")
print(f"\nTime Interval Analysis:")
print(f"  Most common time difference: {most_common_diff}")
print(f"  Time interval: 1 minute (60 seconds)")
print(f"\nMinute values range: {df_sample['minute'].min()} to {df_sample['minute'].max()}")
print(f"Hour values range: {df_sample['hour'].min()} to {df_sample['hour'].max()}")
print(f"\nExpected records per hour: 60 (one per minute)")
print(f"Expected records per day: 1,440 (24 hours × 60 minutes)")

# Check if data is continuous
print(f"\nData Continuity Check:")
print(f"  First timestamp: {df_sample['datetime'].iloc[0]}")
print(f"  Last timestamp (in sample): {df_sample['datetime'].iloc[-1]}")
print(f"  Total records in sample: {len(df_sample)}")
print(f"  Expected records (if continuous): {(df_sample['datetime'].iloc[-1] - df_sample['datetime'].iloc[0]).total_seconds() / 60 + 1:.0f}")

# Count unique time differences
time_diff_counts = df_sample['time_diff'].value_counts().head(10)
print(f"\nTime Difference Distribution (first 10):")
for diff, count in time_diff_counts.items():
    if pd.notna(diff):
        print(f"  {diff}: {count} occurrences")

print("\n" + "="*80)

TIME INTERVAL ANALYSIS

Sample Data Structure:
  Columns: item_id, day, hour, minute, w_ghr, w_gtr, datetime, time_diff

Time Interval Analysis:
  Most common time difference: 0 days 00:01:00
  Time interval: 1 minute (60 seconds)

Minute values range: 0 to 59
Hour values range: 15 to 18

Expected records per hour: 60 (one per minute)
Expected records per day: 1,440 (24 hours × 60 minutes)

Data Continuity Check:
  First timestamp: 2024-04-05 15:00:00
  Last timestamp (in sample): 2024-04-05 18:19:00
  Total records in sample: 200
  Expected records (if continuous): 200

Time Difference Distribution (first 10):
  0 days 00:01:00: 199 occurrences

